# Ray Tasks Revisited

The [Ray Crash Course](../ray-crash-course/00-Ray-Crash-Course-Overview.ipynb) introduced the core concepts of Ray's API and how they parallelize work. Specifically, we learned how to define Ray _tasks_, run them, and retrieve the results. 

This lesson explores Ray tasks in greater depth, including how task dependencies are handled automatically by Ray, `ray.wait()` usage patterns, profiling tasks, and a peak under the hood at Ray's architecture and runtime behavior.

> **Tip:** Recall that the [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

**TODO**: THE REST OF THIS MATERIAL IS THE ORIGINAL CONTENT THAT NEEDS TO BE REFACTORED.

Large actor state and large objects returned from tasks are stored in the object store. Even if a worker goes down but the object store is healthy, the reference to the state/object is lost. (For faster access, smaller objects are not stored in the object store. Instead, they returned to the processes that need them and stored in the corresponding process memory.)


In [2]:
# Imports. We're adding NumPy for the examples and the tutorial `util` library:

import ray, time, sys    # New notebook, so new process
import numpy as np       # Used for examples

In [3]:
def pnd(n, duration, prefix=''):
    """Print an integer and a time duration, with an optional prefix."""
    prefix2 = prefix if len(prefix) == 0 else prefix+' '
    print('{:s}n: {:2d}, duration: {:6.3f} seconds'.format(prefix2, n, duration))

def pd(duration, prefix=''):
    """Print a time duration, with an optional prefix."""
    prefix2 = prefix if len(prefix) == 0 else prefix+' '
    print('{:s}duration: {:6.3f} seconds'.format(prefix2, duration))

In [4]:
ray.init(ignore_reinit_error=True)

2020-04-14 11:06:55,168	INFO resource_spec.py:212 -- Starting Ray with 4.3 GiB memory available for workers and up to 2.17 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-14 11:06:55,554	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:12777',
 'object_store_address': '/tmp/ray/session_2020-04-14_11-06-55_159578_86749/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-14_11-06-55_159578_86749/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-14_11-06-55_159578_86749'}

In [ ]:
print(f'New port? http://{ray.get_webui_url()}')

Let's work with a new remote function. Previously, our `expensive` and `expensive_task` functions returned tuples that included time durations. Obviously the durations were useful for understanding how long the functions took to execute. Now, it will be more convenient to not return "metadata" like this, but just data values that we care about, because we are going to pass them to other functions. 

Hence, we'll define _dependency_ relationships between tasks. We'll learn how Ray handles these dependent, asynchronous computations.

So, let's define a task to return a random NumPy array of some size `n`. As before, we'll add a sleep time, one tenth the size of `n`:

In [5]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [6]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [7]:
start = time.time()
id1 = make_array.remote(20)
id2 = make_array.remote(20)
id3 = add_arrays.remote(id1, id2)
print(ray.get(id3))
pd(time.time() - start, prefix="Total time:")

[-0.72591853  1.48840327 -2.12733342 -0.31191934 -1.85208789  0.69682765
  0.90089066 -1.47122757  0.11244114 -0.11956292  0.0559243  -2.41352552
 -2.99910216  0.80540812  0.40352184  0.51636653  0.16064962  1.48668567
 -0.42439474  1.35801608]
Total time: duration:  4.025 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `id1` and `id2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, _Ray handles task dependencies automatically for us._ 

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Recall from the previous lesson that we explored when to call `ray.get()` to avoid forcing tasks to become synchronous when they should be asynchronous. This additional example illustrates two key points:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

We've seen several examples of the best idiomatic way to use `ray.get()`. Here again is an example from the last lesson:

```python
start = time.time()
ids = [expensive_task.remote(n) for n in range(5)]  # Fire off the asynchronous tasks
for n2, duration in ray.get(ids):                   # Retrieve all the values from the list of futures
    p(n2, duration)
pd(time.time() - start, prefix="Total time:")
```

Let's try it again with our new methods:

In [9]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [-0.23927754 -2.53264946  2.19462732  0.50305556  1.43099456  2.32635644
 -0.87251704 -2.44214929  1.14185185 -0.82872563]
20: [-1.67455833 -1.61799084  2.85896463  0.55199471 -0.71222344 -1.60722795
  1.85160247 -0.25683598  1.94011596 -2.79309293  0.35323319  2.31515346
  1.48841244  2.23044401 -1.45576991  1.35030537 -2.46300856 -3.08309536
 -3.33677276  0.21945927]
30: [ 5.12000021e-01  1.17587428e+00 -8.32026684e-01 -3.36183549e+00
  3.99597448e+00  1.28394987e+00  2.23784551e+00 -6.88538091e-01
  6.19159148e-01  3.61831980e+00  3.92920286e+00 -3.59826796e+00
 -3.23552321e+00 -2.48951110e+00 -1.68175642e+00 -5.02002410e+00
 -1.92775821e-03 -5.26957163e-01  4.22929944e-01 -2.91289456e-01
 -1.32605836e+00  2.69264055e+00  2.74114902e+00 -9.77275983e-01
  3.37298751e+00  2.10068603e+00 -3.69972089e+00  3.79182542e+00
 -3.04859065e+00  4.06552581e-02]
40: [ 0.10159536 -2.22382018 -1.7959269   0.42343121  3.4075889   3.10676476
  2.35631795 -0.29700034  3.36958918  4.05326637

On my machine, I waited 8 seconds and then everything was printed at once.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks might finish quickly, like our contrived examples that sleep for short durations compared to other invocations. 

When you have a list of asynchronous tasks, you want to process the results of them as soon they become available, even while others continue to run. Use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. Now we'll learn how to use these two together. For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

Here is the previous example rewritten to use `ray.wait()`:

In [10]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.004)
0: []
Returned   1 completed tasks. (elapsed time:  2.013)
10: [-0.7748803   2.42292443  1.20948299  4.88852942  0.89837183  1.50937142
 -0.25114321  0.09474296 -0.12395538 -0.20694388]
Returned   1 completed tasks. (elapsed time:  4.014)
20: [ 2.38421976  0.02052721 -3.55827243  3.07740589 -3.4388058  -0.30839728
 -0.58613436  0.89943397  1.08589915 -1.44308556 -1.42154635 -3.74075417
  3.14693995  0.51759549  1.07043034 -1.56425525  3.16214019  0.7590446
  0.94005633 -1.85191739]
Returned   1 completed tasks. (elapsed time:  6.013)
30: [-3.00785216 -0.96698594 -0.8018705  -1.61936126  1.21167068 -0.30249824
 -5.01797047  1.08932897 -0.60273331 -3.41248391 -0.76508517 -0.08940081
 -2.0903908   1.47596597 -0.53615341 -0.29250825  0.03999894  0.64057829
  2.2744951   1.70892307  0.25545052  0.31743777  1.70748236  1.42179621
  1.49232493 -4.50941567  2.73518736  0.26995575 -0.87490048  0.52683519]
Returned   1 completed tasks. (elapse

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the ids that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [11]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.013)
0: []
10: [ 2.48782664  3.60256287  0.76247924 -4.94795021  1.01730811  0.04036607
  1.58268961  3.92500922  0.87885563  1.60555715]
Returned   2 completed tasks. (elapsed time:  6.014)
20: [-1.71401283  5.39790755  2.3582053   0.12227535  2.37604512 -0.48118321
  0.06614954 -0.02731327 -1.77295446 -2.45342676 -0.69967487 -2.2294005
 -2.22775042 -1.73821087 -4.57351573 -0.44137921  2.84857689 -1.54866423
  1.89463067 -2.70276457]
30: [-4.59445258e+00  3.89679505e+00  8.20514801e-01 -5.66513062e-01
 -3.03036166e+00 -1.15683745e+00  3.75271241e+00 -1.80292791e+00
  2.25089150e+00 -3.07337536e-01 -1.97912797e+00  8.48646263e-01
 -1.94561855e+00  4.54889629e+00 -2.92199263e+00  2.65986486e+00
  1.38457939e+00  4.07805802e+00 -1.58673243e+00 -1.31739134e+00
 -4.45172248e+00  2.77553857e-01 -4.55171261e-03 -2.15568289e-01
  2.98708218e+00 -1.56416804e+00  4.58013331e-01 -1.03483589e-01
  1.26595783e+00  1.12236093e+00]
Returned   1 complet

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

## Exercise 1

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

See the [solutions notebook](solutions/Advanced-Ray-Solutions.ipynb) for a discussion.

In [14]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.012)
0: []
10: [ 1.46218396 -1.71564413  2.1042246   0.82173988  1.4533055   1.39600851
 -1.06393577  2.25019116  1.60910142 -0.1514104 ]
Returned   2 completed tasks. (elapsed time:  6.011)
20: [ 2.38725246  2.0657394   0.01455183 -1.45140976 -2.6410359  -3.49991872
 -1.33812099 -0.51134982  1.21788308  0.3857008  -1.50554774  0.38900639
  1.04973992  3.39940876 -0.95476125 -1.84870612 -1.08392993 -0.89986749
 -0.39521986  0.72469544]
30: [ 0.87473995  2.34406588 -0.35945046 -0.26934902 -0.29614885 -0.35242477
  0.02530809 -0.32508638  1.84035711  2.85931701  3.60153247 -3.65024403
  1.72475767  3.7102519  -0.33589828 -0.25583209 -1.00051432  1.27009545
  1.07366468 -2.34082089  0.13948107  1.11422276  1.8402104   2.95700841
  0.47559761 -1.32996335 -3.34878214  0.9953005  -6.04742539  0.67411903]
Returned   1 completed tasks. (elapsed time:  8.012)
40: [-1.21013773 -0.67306055  1.35842755  2.70314894 -0.81763398 -0.21599314
 -1.76389932

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. Don't ask for results you don't need.
> 3. Don't ask for the results you need until you really need them.

## Exercise 3

Let's make sure you understand how to use `ray.wait()`. The definitions from Exercise 1 in the previous lesson are repeated in the next cell. Change the definitions to use Ray. In particular, use `ray.wait()` as we used it above. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

> **Tip:** The solution is in the `solutions` folder.

In [18]:
@ray.remote
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
square_ids = [slow_square.remote(n) for n in range(4)]
while len(square_ids) > 0:
    done_ids, square_ids = ray.wait(square_ids)
    print (f'finished: {ray.get(done_ids)}')
duration = time.time() - start

finished: [0]
finished: [1]
finished: [4]
finished: [9]


In [19]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [20]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [21]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [22]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

7.82012939453125e-05 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [23]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.2821660041809082 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [24]:
start = time.time()
ids = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(ids)
print(f'{time.time() - start} seconds')

0.10722923278808594 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

So, let's run some trials with increasingly large array sizes, to compare the performance with local vs. remote execution. First, we'll set up `matplotlib`:

In [25]:
local_durations = []
remote_durations = []
# These n values were determined by experimentation on this test machine. 
# If you are using an old machine, and this cell takes a long time to execute,
# you could set the `trials` value above to a smaller number. 
ns = [i*(10**j) for j in range(2,5) for i in [1,2,3,5,8]]
for n in ns:
    start_local = time.time()
    [local_make_array(n) for _ in range(trials)]
    local_durations.append(time.time() - start_local)
    
    start_remote = time.time()
    ids = [remote_make_array.remote(n) for _ in range(trials)]
    ray.get(ids)
    remote_durations.append(time.time() - start_remote)
(ns, local_durations, remote_durations)

([100,
  200,
  300,
  500,
  800,
  1000,
  2000,
  3000,
  5000,
  8000,
  10000,
  20000,
  30000,
  50000,
  80000],
 [0.0013120174407958984,
  0.0009620189666748047,
  0.0010399818420410156,
  0.0015048980712890625,
  0.0026543140411376953,
  0.002874135971069336,
  0.007361888885498047,
  0.010944128036499023,
  0.01852273941040039,
  0.030078887939453125,
  0.03212404251098633,
  0.061463117599487305,
  0.08843016624450684,
  0.14572501182556152,
  0.23099613189697266],
 [0.026623964309692383,
  0.02771472930908203,
  0.023834705352783203,
  0.022733211517333984,
  0.026467084884643555,
  0.0232388973236084,
  0.0239717960357666,
  0.02183389663696289,
  0.04171013832092285,
  0.026540756225585938,
  0.030015945434570312,
  0.04691290855407715,
  0.06048893928527832,
  0.07406997680664062,
  0.08021998405456543])

In [26]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

tooltips = [
    ("name", "$name"),
    ("array size", "$x"),
    ("time", "$y")]
p1 = figure(x_axis_type="log", y_axis_type="log", title="Execution Times", tooltips=tooltips)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'array size'
p1.yaxis.axis_label = 'time'

p1.line(ns, local_durations, color='#A6CEE3', legend_label='local', name='local')
p1.circle(ns, local_durations, color='darkgrey', size=4)
p1.line(ns, remote_durations, color='#B2DF8A', legend_label='remote', name='remote')
p1.square(ns, remote_durations, color='darkgrey', size=4)
p1.legend.location = "top_left"

show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

Here's a static image from a test run, in case the Bokeh plot isn't working. Your results may look a lot different!
![Execution Times: Local versus Remote](../images/Execution-Times-Local-v-Remote.png)

Let's confirm what the graph shows as the crossing point:

In [27]:
i=0
while i < len(ns) and local_durations[i] < remote_durations[i]:
    i=i+1
print('The Ray times are faster starting at n = {:d}, local = {:6.3f} vs. remote = {:6.3f}'.format(
    ns[i], local_durations[i], remote_durations[i]))

The Ray times are faster starting at n = 8000, local =  0.030 vs. remote =  0.027


## How Distributed Task Management Works

> **Note:** If you just want to learn the Ray API, you can safely skip the rest of this lesson (notebook) for now. It continues the exploration of how Ray works internally, which we started in the previous lesson. However, you should come back to this material at some point, so you'll develop a better understanding of how Ray works.

At the end of the last lesson, we examined Ray task scheduling at a high-level, by watching the Ray Dashboard and analyzing the performance times. Now we'll walk through some images that show the process Ray follows to place tasks around a cluster. 

Assume we will invoke the `make_array` task twice, then invoke `add_arrays` to sum the returned NumPy arrays. Graphically, it looks as follows:

![Ray under the hood 1](../images/Ray-Cluster/Ray-Cluster.001.jpeg)

How does this get scheduled in a cluster? Here we'll assume a three-node cluster that has resources for running two Ray worker tasks per node (under powered compared to what we learned using Ray Dashboard last lesson!).
![Ray under the hood 2](../images/Ray-Cluster/Ray-Cluster.002.jpeg)

First, assume that the driver program is running on Node1. So it will invoke the local scheduler to schedule the three tasks.
![Ray under the hood 3](../images/Ray-Cluster/Ray-Cluster.003.jpeg)

Immediately the ids for the task futures are returned. The _Global Control Store_ tracks where every task is running and every object is stored in the local _Object Stores_.
![Ray under the hood 4](../images/Ray-Cluster/Ray-Cluster.004.jpeg)

Suppose the local scheduler has available capacity in the first worker on the same node. It schedules the first `make_array` task there.
![Ray under the hood 5](../images/Ray-Cluster/Ray-Cluster.005.jpeg)

It decides to schedule the second `make_array` task in a worker on node 2.
![Ray under the hood 6](../images/Ray-Cluster/Ray-Cluster.006.jpeg)

When the two tasks finish, they place their result objects in their local object stores.
![Ray under the hood 7](../images/Ray-Cluster/Ray-Cluster.007.jpeg)

Now `add_array` can be scheduled, because the two tasks it depends on are done. Let's suppose it gets scheduled in the second worker on Node 1.
![Ray under the hood 8](../images/Ray-Cluster/Ray-Cluster.008.jpeg)

The first object it needs is already on the same node, in the object store, so the `add_arrays` task can _read it directly from shared memory_. No copying is required to the worker's process space.
![Ray under the hood 9](../images/Ray-Cluster/Ray-Cluster.009.jpeg)

However, the second object is on a different node, so Ray copies it to the local object store. 
![Ray under the hood 10](../images/Ray-Cluster/Ray-Cluster.010.jpeg)

Now it can also be read from shared memory.
![Ray under the hood 11](../images/Ray-Cluster/Ray-Cluster.011.jpeg)

When `add_arrays` is finished, it writes its results to the local object store.
![Ray under the hood 12](../images/Ray-Cluster/Ray-Cluster.012.jpeg)

At this point, if the driver calls `ray.get(id3)`, it will return `obj3`.
![Ray under the hood 13](../images/Ray-Cluster/Ray-Cluster.013.jpeg)

Whew! Hopefully you have a better sense of what Ray does under the hood. Scheduling tasks on other nodes and copying objects between object stores is efficient, but incurs unavoidable network overhead.